In [9]:
import os
import pickle
import pprint
from pathlib import Path
from collections import OrderedDict, defaultdict
from easydict import EasyDict

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import torch
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader
from tqdm import tqdm

from batch_engine import valid_trainer, batch_trainer
from config import argument_parser
from dataset.AttrDataset import AttrDataset, get_transform
from loss.CE_loss import CEL_Sigmoid
from models.base_block import FeatClassifier, BaseClassifier
from models.resnet import resnet50, resnet101

from tools.function import get_model_log_path, get_pedestrian_metrics
from tools.utils import time_str, save_ckpt, ReDirectSTD, set_seed, AverageMeter
from tools.attributes_dict import attr_list

import sys
sys.path.append("/home/pantengteng/Programs/Yet-Another-EfficientDet-Pytorch/")
from TensorRT_python.src.NetWork import *

In [10]:
args = EasyDict()
args.dataset = "test"
args.redirector = True
# args.device
args.valid_split = "test"
args.test_batchsize = 1
args.attr_num = 48
args.height = 256
args.width = 192
args.threshold = 0.5
# args.pretrained_model = '/home/pantengteng/Programs/Strong_Baseline_of_Pedestrian_Attribute_Recognition/exp_result/PETA/2020-12-10_16:17:39_ep_30_bs_64/img_model/ckpt_max.pth' 
# args.pretrained_model = '/home/pantengteng/Programs/Strong_Baseline_of_Pedestrian_Attribute_Recognition/exp_result/PA100k/2020-12-10_20:28:54_ep_30_bs_64/img_model/ckpt_max.pth' 
args.pretrained_model = '/home/pantengteng/Programs/Strong_Baseline_of_Pedestrian_Attribute_Recognition/exp_result/combined/2020-12-12_11:42:11_ep_30_bs_64/img_model/ckpt_max.pth'

In [11]:
def get_reload_weight(model_path, model):
    model_dict = torch.load(model_path)
    from collections import OrderedDict
    new_state_dict = OrderedDict()
    for k, v in model_dict["state_dicts"].items():
        name = k[7:]  # 去掉 `module.`
        if name not in model.state_dict():
            continue
        new_state_dict[name] = v
    model.load_state_dict(new_state_dict)
    return model

_, valid_tsfm = get_transform(args)

valid_set = AttrDataset(args=args, split=args.valid_split, transform=valid_tsfm)

valid_loader = DataLoader(
    dataset=valid_set,
    batch_size=args.test_batchsize,
    shuffle=False,
    num_workers=1,
    pin_memory=True,
)

print(f'{args.valid_split} set: {len(valid_loader.dataset)}, '
      f'attr_num : {args.attr_num}')

backbone = resnet50()
classifier = BaseClassifier(nattr=args.attr_num)
model = FeatClassifier(backbone, classifier).cuda()

print("reloading pretrained models")

model_path = args.pretrained_model
model = get_reload_weight(model_path, model)
# model.classifier.fuse()

valid_iter = iter(valid_loader)
input_data,_,_ = next(valid_iter)
input_data = input_data.cuda()

# export
torch.onnx.export(model,                     # model being run
                  input_data,                # model input (or a tuple for multiple inputs)
                  "par_test_rbn.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=11,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  verbose=True,
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  #dynamic_axes={'input' : {0 : 'batch_size'},    # variable lenght axes
                  #             'output' : {0 : 'batch_size'}}
                 )

test set: 140, attr_num : 48
reloading pretrained models
graph(%input : Float(1:147456, 3:49152, 256:192, 192:1),
      %backbone.conv1.weight : Float(64:147, 3:49, 7:7, 7:1),
      %backbone.bn1.weight : Float(64:1),
      %backbone.bn1.bias : Float(64:1),
      %backbone.bn1.running_mean : Float(64:1),
      %backbone.bn1.running_var : Float(64:1),
      %backbone.layer1.0.conv1.weight : Float(64:64, 64:1, 1:1, 1:1),
      %backbone.layer1.0.bn1.weight : Float(64:1),
      %backbone.layer1.0.bn1.bias : Float(64:1),
      %backbone.layer1.0.bn1.running_mean : Float(64:1),
      %backbone.layer1.0.bn1.running_var : Float(64:1),
      %backbone.layer1.0.conv2.weight : Float(64:576, 64:9, 3:3, 3:1),
      %backbone.layer1.0.bn2.weight : Float(64:1),
      %backbone.layer1.0.bn2.bias : Float(64:1),
      %backbone.layer1.0.bn2.running_mean : Float(64:1),
      %backbone.layer1.0.bn2.running_var : Float(64:1),
      %backbone.layer1.0.conv3.weight : Float(256:64, 64:1, 1:1, 1:1),
      %ba

In [9]:
model.eval()
valid_logits = model(input_data)
valid_probs = torch.sigmoid(valid_logits)
import onnxruntime as ort
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

ort_session = ort.InferenceSession('par.onnx') # _sigmoid

ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(input_data)}

ort_outs = ort_session.run(None, ort_inputs)

print(ort_outs[0], valid_logits)

# compare ONNX Runtime and PyTorch results
np.testing.assert_allclose(to_numpy(valid_logits), ort_outs[0], rtol=1e-03, atol=1e-05)

print("Exported model has been tested with ONNXRuntime, and the result looks good!")

[[0.00213012 0.00123104 0.98306394 0.0271132  0.01300848 0.9948066
  0.01440483 0.00273395 0.00254059 0.00742626 0.00923717 0.06284043
  0.99834144 0.00171646 0.02653241 0.00204554 0.00323388 0.9868541
  0.00733489 0.00198856 0.03633046 0.00652286 0.00770414 0.6637916
  0.00478259 0.00331512 0.28779656 0.00490484 0.05462739 0.03829703
  0.00649369 0.10182652 0.00681683 0.9987594  0.01793328 0.0093475
  0.00528982 0.0318253  0.00227207 0.00578701 0.02111682 0.94216114
  0.00491497 0.00328514 0.00630468 0.0049201  0.07536027 0.06937131]] tensor([[0.0021, 0.0012, 0.9831, 0.0271, 0.0130, 0.9948, 0.0144, 0.0027, 0.0025,
         0.0074, 0.0092, 0.0628, 0.9983, 0.0017, 0.0265, 0.0020, 0.0032, 0.9869,
         0.0073, 0.0020, 0.0363, 0.0065, 0.0077, 0.6638, 0.0048, 0.0033, 0.2878,
         0.0049, 0.0546, 0.0383, 0.0065, 0.1018, 0.0068, 0.9988, 0.0179, 0.0093,
         0.0053, 0.0318, 0.0023, 0.0058, 0.0211, 0.9422, 0.0049, 0.0033, 0.0063,
         0.0049, 0.0754, 0.0694]], device='cuda:0', g

In [125]:
valid_result = get_pedestrian_metrics(gt_label, preds_probs, threshold=args.threshold)

print(f'Evaluation on test set, \n',
      'ma: {:.4f},  pos_recall: {:.4f} , neg_recall: {:.4f} \n'.format(
          valid_result.ma, np.mean(valid_result.label_pos_recall), np.mean(valid_result.label_neg_recall)),
      'Acc: {:.4f}, Prec: {:.4f}, Rec: {:.4f}, F1: {:.4f}'.format(
          valid_result.instance_acc, valid_result.instance_prec, valid_result.instance_recall,
          valid_result.instance_f1))

In [6]:
# convert trt engine
def preprocess(image_path, size, mean=(0.406, 0.456, 0.485), std=(0.225, 0.224, 0.229)):
    ori_img = [cv2.imread(img_path) for img_path in image_path]
    normalized_img = (ori_img / 255 - mean) / std
    resized_img = cv2.resize(normalized_img, size)

    return img[..., ::-1]
class Process():
    def __init__(self, size = (192, 256)):
        self.size = size

    def run(self, calib_data):
        data = []
        list_img = os.listdir(calib_data)
        import pdb; pdb.set_trace()
        for file_name in list_img:
            img= preprocess(file_name,size = self.size)
            data.append(img)
        data = np.array(data)
        return data

In [7]:
pprocess = Process()
net = NetWork('par_rbn.onnx',
            'fp32',
            calib_data='/home/pantengteng/Programs/Strong_Baseline_of_Pedestrian_Attribute_Recognition/data/test/images',
            preprocessing=pprocess,
            cache_file= 'calib_data',
            batch_size=1
            )
preds_probs = []
gt_list = []

for step, (imgs, gt_label, imgname) in enumerate(tqdm(valid_loader)):
    gt_list.append(gt_label.cpu().numpy())
    valid_logits = net.inference(imgs.numpy())
    preds_probs.append(valid_logits)
gt_label = np.concatenate(gt_list, axis=0)
preds_probs = np.concatenate(preds_probs, axis=0)

trt_valid_result = get_pedestrian_metrics(gt_label, preds_probs, threshold=args.threshold)

print(f'Evaluation on test set, \n',
      'ma: {:.4f},  pos_recall: {:.4f} , neg_recall: {:.4f} \n'.format(
          valid_result.ma, np.mean(valid_result.label_pos_recall), np.mean(valid_result.label_neg_recall)),
      'Acc: {:.4f}, Prec: {:.4f}, Rec: {:.4f}, F1: {:.4f}'.format(
          valid_result.instance_acc, valid_result.instance_prec, valid_result.instance_recall,
          valid_result.instance_f1))

Loading ONNX file from path par_rbn.onnx...
Beginning ONNX file parsing
Completed parsing of ONNX file
Building an engine from file par_rbn.onnx; this may take a while...
Completed creating Engine


AttributeError: 'NoneType' object has no attribute 'serialize'